In [1]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression, Ridge


In [2]:
p = 7
n = 10
min_cor = 0
max_cor = 0.1

true_betas = np.array([0,1,2,3,4,5,6])

In [3]:
def get_sim_data(p, n, min_cor, max_cor, true_betas):
    
    sd_vec = np.ones(p) 
    mean = np.zeros(p)
    cor_matrix = np.zeros((p,p))

    correlation = np.random.uniform(min_cor, max_cor, int(p * (p - 1) / 2))
    cor_matrix[np.triu_indices(p, 1)] = correlation
    cor_matrix[np.tril_indices(p, -1)] = cor_matrix.T[np.tril_indices(p, -1)]
    np.fill_diagonal(cor_matrix, 1)


    D = np.diag(sd_vec)
    sigma = D.dot(cor_matrix).dot(D)

    X = np.random.multivariate_normal(mean, sigma, n)
    eps = np.random.normal(0, 1, n)

    y = X.dot(true_betas) + eps 
    
    y = pd.Series(y, name = "y")
    
    column_names = []
    
    for value in range(1, p + 1): 
        
        column = f"X_{value}"
        column_names.append(column)
        
    
    X = pd.DataFrame(X, columns = column_names)
    
    df = pd.concat([y, X], axis = 1)
    
    return y, X, df



In [4]:
y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas)

In [5]:
df

,y,X_1,X_2,X_3,X_4,X_5,X_6,X_7
0,-1.493512,0.632404,1.404622,-0.190649,1.219714,-0.294345,-1.152318,0.110566
1,3.295121,0.555723,1.053443,-0.114893,-1.661567,0.079000,1.305844,0.170367
2,-3.610267,1.906740,0.610911,-1.551280,-0.252927,0.587322,-0.219268,-0.205094
3,3.298628,-0.800948,1.153739,0.176091,0.408189,0.670084,0.414627,-0.677825
4,-6.762056,-0.027841,-1.806505,-0.449488,-0.556315,-1.196378,0.960951,-0.518539
5,-7.691519,-0.915100,-0.425751,-0.508769,-0.706509,-2.017201,0.952909,-0.123409
6,5.616446,-1.200474,0.262506,1.123411,0.332610,1.703207,-0.612327,0.018954
7,9.691238,1.627276,0.311077,1.023488,-0.954175,1.255641,1.171743,-0.072876
8,-18.744499,-1.518489,-0.374089,-0.818530,-0.983959,-1.202199,0.471063,-1.883018
9,-6.208691,0.235632,-0.401799,-0.576948,-1.836569,-0.127569,-0.667782,0.564086


In [6]:
reg = LinearRegression().fit(X, y)

In [7]:
reg.coef_


array([0.35139125, 0.89232976, 2.22264483, 2.6458013 , 3.47480882,
       4.54774043, 5.84639729])

In [8]:
"""Zero multicollinearity.
   n = 30; p = 2, 28, 30, 35. 
"""
np.random.seed(123)

n_sim = 30
p_sim = np.array([2, 28, 30, 35])
dataframes = len(p_sim)
min_corr_sim = 0
max_corr_sim = 0 
iterations_sim = 500

matrix_betas_final = []
matrix_betas_mean = []
matrix_bias_sq = []
matrix_betas_var_mean = []
matrix_betas_var_final = []


for m in range(dataframes): 
    
    
    for p in p_sim: 
    
        matrix_var = []
        matrix_betas = []
    
        for i in range(iterations_sim):
    
            true_betas_sim = np.repeat(5, p)
    
            y, X, df = get_sim_data(p, n_sim, min_corr_sim, max_corr_sim, true_betas_sim) 
    
            var_cov = np.linalg.inv(np.dot(X.T, X))
            var_betas = var_cov.diagonal() 
        
            #reg = LinearRegression().fit(X, y)
            betas = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
        
            #matrix_betas.append(reg.coef_)
            matrix_betas.append(betas)
        
            matrix_var.append(var_betas)
        
            X_names = []
            beta_names = []
    
            for value in range(1, p + 1): 
            
                column_X = f"X_{value}"
                column_betas = f"beta_{value}"
                X_names.append(column_X)
                beta_names.append(column_betas)
        
            df_var = pd.DataFrame(matrix_var, columns = X_names)
            df_betas = pd.DataFrame(matrix_betas, columns = beta_names) 
        
            bias_squared = (df_betas.mean() - true_betas_sim)**2
        
        matrix_betas_final.append(df_betas)
        matrix_betas_mean.append(df_betas.mean())
        matrix_bias_sq.append(bias_squared)
        matrix_betas_var_mean.append(df_var.mean())
        matrix_betas_var_final.append(df_var)
    
 

In [9]:
matrix_betas_var_mean[0]
matrix_betas_var_mean[1]
matrix_betas_var_mean[2]
matrix_betas_var_mean[3]

X_1    -2.766848e+13
X_2    -3.320289e+13
X_3     3.246389e+13
X_4     6.095380e+12
X_5    -2.690809e+13
X_6    -3.741737e+13
X_7    -2.049798e+13
X_8    -4.448690e+13
X_9    -1.379433e+13
X_10   -2.104500e+13
X_11    1.082571e+13
X_12    2.780041e+12
X_13    9.565688e+11
X_14   -2.279610e+12
X_15    4.250038e+13
X_16    6.405215e+12
X_17   -1.033276e+13
X_18   -4.202050e+13
X_19   -1.930038e+13
X_20   -2.906674e+13
X_21    2.711959e+13
X_22    1.142487e+13
X_23   -9.667820e+13
X_24   -3.028840e+12
X_25   -3.052798e+13
X_26   -1.196195e+13
X_27    1.181959e+13
X_28   -2.375272e+13
X_29   -5.396132e+13
X_30   -1.359054e+13
X_31   -3.697549e+13
X_32   -1.530770e+13
X_33    6.374014e+12
X_34   -1.058066e+13
X_35    4.707655e+12
dtype: float64

In [9]:
np.random.seed(159)

p = 28
n = 30
true_betas = np.repeat(5, p)
min_cor = 0 
max_cor = 0 
alpha = 0.001


y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas)

n, m = X.shape
I = np.identity(m)


ols = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)

ols_var_cov = np.linalg.inv(np.dot(X.T, X))
ols_var = ols_var_cov.diagonal()

alphas = np.array([0, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1, 2, 3, 5, 10, 15, 20])

matr_var = []
matr_beta = []

for a in alphas: 

    ridge_beta = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), X.T), y)
    matr_beta.append(ridge_beta)
    
    ridge_var_cov = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), np.dot(X.T, X)), np.linalg.inv(np.dot(X.T, X) + a * I))
    ridge_var = ridge_var_cov.diagonal()
    matr_var.append(ridge_var)

beta_var_names = []
ridge_beta_names = []
    
for value in range(1, p + 1): 
    
    column_betas_var = f"beta_var_{value}"
    column_betas = f"beta_{value}"
    beta_var_names.append(column_betas_var)
    ridge_beta_names.append(column_betas)


In [10]:
beta_var = pd.DataFrame(matr_var, columns = beta_var_names)
beta_var["alpha"] = alphas
beta_var.set_index("alpha")    

,beta_var_1,beta_var_2,beta_var_3,beta_var_4,beta_var_5,beta_var_6,beta_var_7,beta_var_8,beta_var_9,beta_var_10,...,beta_var_189,beta_var_190,beta_var_191,beta_var_192,beta_var_193,beta_var_194,beta_var_195,beta_var_196,beta_var_197,beta_var_198
alpha,,,,,,,,,,,,,,,,,,,,,
0.0000,2.427521,0.663690,1.027757,1.741210,8.652040,0.705819,1.167791,0.209445,0.475403,1.731693,...,0.464720,1.398587,0.242995,4.707146,5.263580,0.286077,0.432695,0.264678,2.052473,2.387135
0.0001,2.268247,0.638268,1.015365,1.677230,8.118124,0.696873,1.102453,0.208107,0.460090,1.643960,...,0.463072,1.368871,0.240256,4.421376,4.932370,0.284450,0.421808,0.254588,1.924919,2.358828
0.0010,1.365506,0.489142,0.920092,1.289592,5.063371,0.636114,0.727835,0.198151,0.372755,1.134611,...,0.451391,1.173477,0.223555,2.786644,3.046089,0.271885,0.354129,0.194701,1.197613,2.140829
0.0100,0.233427,0.236424,0.484755,0.491798,0.885691,0.419897,0.204628,0.153566,0.250316,0.344555,...,0.389761,0.584647,0.184876,0.552214,0.567427,0.210356,0.193110,0.085665,0.232947,1.135968
0.1000,0.071878,0.109102,0.109703,0.102415,0.102458,0.134093,0.083359,0.087886,0.147321,0.092087,...,0.171041,0.131156,0.122357,0.108965,0.119170,0.112952,0.080600,0.037393,0.067317,0.221369
0.2000,0.052158,0.083912,0.072989,0.070817,0.071605,0.079757,0.069978,0.068406,0.105920,0.066938,...,0.106042,0.082435,0.096294,0.079727,0.078798,0.084015,0.060815,0.032334,0.055253,0.124536
0.5000,0.033333,0.055552,0.044977,0.045025,0.047987,0.041008,0.053829,0.046834,0.062048,0.044307,...,0.054870,0.048374,0.064342,0.053194,0.045295,0.051499,0.040217,0.026727,0.040510,0.059026
1.0000,0.024475,0.038056,0.032094,0.032398,0.033820,0.026810,0.041855,0.034336,0.040391,0.032550,...,0.035335,0.034211,0.044444,0.037953,0.030069,0.033482,0.029148,0.022248,0.029858,0.035925
2.0000,0.018267,0.024648,0.022964,0.023233,0.022680,0.018520,0.030327,0.024222,0.026329,0.023708,...,0.023922,0.024243,0.028791,0.026096,0.020006,0.021458,0.021236,0.017511,0.020976,0.023199


In [11]:
ridge_beta = pd.DataFrame(matr_beta, columns = ridge_beta_names)
ridge_beta["alpha"] = alphas
ridge_beta.set_index("alpha")

,beta_1,beta_2,beta_3,beta_4,beta_5,beta_6,beta_7,beta_8,beta_9,beta_10,...,beta_189,beta_190,beta_191,beta_192,beta_193,beta_194,beta_195,beta_196,beta_197,beta_198
alpha,,,,,,,,,,,,,,,,,,,,,
0.0000,1.431469,5.807467,3.926156,2.236071,0.072503,7.624101,7.220508,5.102859,4.031463,1.447353,...,4.220671,5.882952,5.872391,8.546182,1.462723,4.922933,4.285627,4.374181,1.437395,5.958386
0.0001,1.551141,5.760691,3.915839,2.308962,0.281415,7.600136,7.147344,5.096939,4.067559,1.535833,...,4.228182,5.847133,5.857172,8.392787,1.628563,4.918490,4.315292,4.401010,1.545047,5.926595
0.0010,2.338443,5.451785,3.858992,2.793782,1.640578,7.438045,6.670658,5.058287,4.303149,2.121703,...,4.274300,5.623312,5.755458,7.393551,2.708076,4.891785,4.513975,4.574284,2.255773,5.701470
0.0100,4.108101,4.753317,3.948965,3.958287,4.418631,6.995541,5.674130,4.953750,4.783558,3.498915,...,4.297046,5.352289,5.495505,5.317081,4.910800,4.871660,5.022244,4.908232,3.901663,4.875434
0.1000,4.758536,4.820550,4.730247,4.126283,4.941148,6.713544,5.227479,4.417448,4.578426,3.955284,...,3.730093,6.021328,5.257863,4.616056,5.022495,4.828649,5.279141,4.975867,4.645925,3.368100
0.2000,4.744429,5.063842,5.059990,3.865562,4.874784,6.685456,5.036722,4.032856,4.343217,3.838261,...,3.357149,6.355777,5.131745,4.428845,4.718949,4.763723,5.262315,4.970348,4.674205,2.772379
0.5000,4.603572,5.383110,5.408980,3.515787,4.777159,6.622313,4.653210,3.432380,4.030708,3.569397,...,2.814711,6.746293,4.826490,4.070268,4.211372,4.675612,5.206396,4.864943,4.557786,2.034509
1.0000,4.401496,5.517441,5.474450,3.388484,4.722273,6.515898,4.239832,2.938190,3.873702,3.321878,...,2.446932,6.900269,4.468958,3.700577,3.778271,4.617793,5.180779,4.629816,4.349650,1.661489
2.0000,4.120503,5.530916,5.266060,3.435371,4.661601,6.315323,3.701556,2.370742,3.801054,3.054539,...,2.126772,6.891306,4.039137,3.245606,3.289118,4.547337,5.215155,4.207783,4.066485,1.514109


$$
V\left(\hat{\boldsymbol{\beta}}^{*}\right)=\sigma^{2}\left[\mathbf{X}^{\prime} \mathbf{X}+k \mathbf{I}\right]^{-1}\left(\mathbf{X}^{\prime} \mathbf{X}\right)\left[\mathbf{X}^{\prime} \mathbf{X}+k \mathbf{I}\right]^{-1}.
$$

In [23]:
np.random.seed(159)

p = 28
n = 30
min_cor = 0 
max_cor = 0 
iterations_sim = 100

alphas = np.array([0, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1, 2, 3, 5, 10, 15, 20])


matrix_ridge_betas = []
matrix_ridge_var = []


for a in alphas: 
    
    matr_var = []
    matr_beta = []
    
    for i in range(iterations_sim):
        
        true_betas_sim = np.repeat(5, p)
    
        y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas_sim) 
        
        n, m = X.shape
        I = np.identity(m)

        ridge_beta = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), X.T), y)
        matr_beta.append(ridge_beta)
        df_ridge_betas = pd.DataFrame(matr_beta).mean()
        
        ridge_var_cov = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), np.dot(X.T, X)), np.linalg.inv(np.dot(X.T, X) + a * I))
        ridge_var = ridge_var_cov.diagonal()
        matr_var.append(ridge_var)
        df_ridge_var = pd.DataFrame(matr_var).mean()
    
    matrix_ridge_betas.append(df_ridge_betas)
    df_ridge_betas = pd.DataFrame(matrix_ridge_betas)
    
    matrix_ridge_var.append(df_ridge_var)
    df_ridge_var = pd.DataFrame(matrix_ridge_var)
    
    
    
    
    
    #for value in range(1, p + 1): 
    
        #column_betas_var = f"beta_var_{value}"
        #column_betas = f"beta_{value}"
        #beta_var_names.append(column_betas_var)
        #ridge_beta_names.append(column_betas)




In [22]:
df_ridge_betas

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,4.631966,5.009964,5.461153,4.727924,5.170259,4.992243,5.025269,5.414854,4.905098,5.123166,...,5.280869,4.798213,4.810379,4.917640,4.879084,4.540000,4.601351,5.034101,4.950184,5.241284
1,4.439592,5.093089,5.482007,4.875797,4.941902,5.130247,4.921094,4.763550,4.838509,5.279714,...,5.288814,4.738743,4.644483,5.267653,4.823515,4.880820,4.495276,4.563793,5.062985,5.215154
2,4.972731,5.187473,4.709316,4.825474,5.171433,5.592231,4.507625,4.888311,4.703277,4.913906,...,5.056799,4.595083,4.932613,4.551067,4.698296,5.524103,4.767068,4.899650,4.664042,5.548234
3,5.210952,5.106639,4.821546,5.193399,5.495535,4.871443,5.296240,5.187799,5.116308,4.614272,...,4.760831,4.673499,5.188478,4.851760,4.721466,4.634540,4.882268,4.684347,4.719396,5.359145
4,4.274590,4.840592,4.839986,5.370642,4.821879,5.129052,4.976447,4.888179,4.836072,4.226115,...,4.821443,5.268943,5.184771,5.135245,4.993625,4.691748,4.715904,4.611014,4.536999,5.533498
5,4.874917,4.954007,5.058572,4.718979,5.073640,5.170374,4.702563,4.334072,4.673532,4.637316,...,4.542682,4.331997,4.679829,4.957075,4.959515,4.963546,4.390012,5.048032,4.432885,4.918549
6,4.589982,4.482726,3.834016,4.136201,4.508151,3.851540,2.818875,3.731024,4.665222,4.201497,...,4.668741,4.031405,3.936035,4.402657,3.917260,3.987461,3.682678,4.084182,4.352638,3.989021
7,3.421758,4.547411,4.851295,4.088233,3.868828,4.263445,3.488521,4.364423,4.242860,4.884753,...,4.572615,4.661431,3.797332,4.368853,4.038808,4.354229,3.964440,4.976500,4.517577,4.386164
8,3.584425,4.793920,3.950672,3.681696,2.923571,3.853587,4.191983,4.537200,4.183027,3.409430,...,4.418312,3.860370,3.453428,3.761790,4.747052,3.820638,3.571671,2.778479,2.843783,5.165965
9,3.791112,3.679311,4.690962,4.032820,2.576608,4.469280,4.108588,3.486887,2.936456,4.836733,...,2.709970,3.100893,3.348780,3.780826,3.903134,2.849773,3.996375,4.253691,4.185184,3.298329


In [24]:
df_ridge_var

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.957375,0.688845,0.637817,0.577718,0.752245,0.730127,0.797852,0.732711,0.712207,0.818806,...,0.681773,0.787141,0.734623,0.673139,0.627051,0.768007,0.693829,0.682652,0.839513,0.869457
1,1.256645,0.989779,1.935547,1.063626,2.023294,0.811360,2.707119,1.259136,1.098345,0.899570,...,1.366354,0.935292,1.218902,1.283763,0.963253,1.302415,2.053522,1.485350,0.997775,1.065217
2,0.742783,0.741809,1.025971,0.852625,0.582436,0.806204,0.706346,0.938637,0.832719,0.904777,...,1.086684,0.702153,0.899858,0.693709,0.960827,0.865547,0.993776,1.033680,0.964384,0.722053
3,0.607854,0.581948,0.607734,0.604471,0.521129,0.605945,0.568279,0.686710,0.547713,0.627673,...,0.628842,0.687882,0.580697,0.720455,0.447756,0.591910,0.565344,0.532423,0.741426,0.541398
4,0.276902,0.264562,0.250375,0.275425,0.270588,0.242730,0.251242,0.270744,0.221052,0.273110,...,0.273491,0.244446,0.253698,0.228693,0.262646,0.237922,0.274738,0.259168,0.251184,0.240991
5,0.181870,0.183934,0.180545,0.187605,0.188868,0.173695,0.185574,0.185853,0.177167,0.190920,...,0.183290,0.187909,0.180180,0.196415,0.178167,0.193309,0.180565,0.172554,0.186502,0.176832
6,0.115433,0.115486,0.112639,0.117474,0.111967,0.116235,0.114910,0.114172,0.108847,0.115025,...,0.121080,0.116543,0.115091,0.118608,0.118174,0.115324,0.118022,0.109336,0.112588,0.108803
7,0.077702,0.077646,0.077240,0.077846,0.079067,0.078052,0.080095,0.079081,0.076776,0.073556,...,0.075137,0.078297,0.080491,0.076517,0.076201,0.078991,0.076744,0.076794,0.078459,0.078563
8,0.050370,0.051604,0.052106,0.051122,0.049477,0.051472,0.051726,0.049864,0.051512,0.050377,...,0.053007,0.051214,0.048652,0.049501,0.051018,0.052319,0.049389,0.048622,0.051216,0.050514
9,0.039746,0.039116,0.038905,0.040095,0.038403,0.038702,0.038340,0.039777,0.038518,0.038269,...,0.038763,0.040017,0.038791,0.038782,0.038667,0.037844,0.037920,0.038280,0.038638,0.039852
